# **Trabalho Final Spark com dataset iris.csv**


Este trabalho busca implementar um algoritmo de machine learning, utilizando a técnica de regressão logística para um problema de classificação.

O dataset utilizado é o íris.csv. Este dataset é um dataset de flores que possui três espécies: iris-setosa, iris-versicolor e iris-virginica. Para cada linha do dataset tem-se as medidas do comprimento e a largura das sépalas e pétalas, em centímetros.

### Importação e Configuração do ambiente Spark

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Importação dos pacotes de contexto e configuração do Spark
from pyspark.sql import SparkSession

In [3]:
# Montando sistema de arquivo com Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# Pasta que contém arquivos utilizados na disciplina e trabalho final
import os
os.chdir("/content/drive/My Drive/Colab Notebooks/Spark")

In [5]:
# mostra arquivo iris.csv, utilizado aqui
!ls

auto-miles-per-gallon.csv  iris.csv	   medicos.csv
genre.csv		   listenings.csv  saudeja.csv


### Setando a configuração do Spark Context

In [6]:
spark = SparkSession.builder.appName("Iris").master("local[*]").getOrCreate()

### Carregando arquivo iris.csv

In [7]:
df_iris = spark.read.csv(path='iris.csv', sep=',',
# encoding='UTF-8',
# comment=None,
header=True,
inferSchema=True)

### Mostrando primeiras linhas

In [8]:
df_iris.show()

+-------------+------------+-------------+------------+-----------+
|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|    Species|
+-------------+------------+-------------+------------+-----------+
|          5.1|         3.5|          1.4|         0.2|Iris-setosa|
|          4.9|         3.0|          1.4|         0.2|Iris-setosa|
|          4.7|         3.2|          1.3|         0.2|Iris-setosa|
|          4.6|         3.1|          1.5|         0.2|Iris-setosa|
|          5.0|         3.6|          1.4|         0.2|Iris-setosa|
|          5.4|         3.9|          1.7|         0.4|Iris-setosa|
|          4.6|         3.4|          1.4|         0.3|Iris-setosa|
|          5.0|         3.4|          1.5|         0.2|Iris-setosa|
|          4.4|         2.9|          1.4|         0.2|Iris-setosa|
|          4.9|         3.1|          1.5|         0.1|Iris-setosa|
|          5.4|         3.7|          1.5|         0.2|Iris-setosa|
|          4.8|         3.4|          1.6|      

### Imprimindo o esquema de iris.csv : 


In [9]:
df_iris.printSchema()

root
 |-- SepalLengthCm: double (nullable = true)
 |-- SepalWidthCm: double (nullable = true)
 |-- PetalLengthCm: double (nullable = true)
 |-- PetalWidthCm: double (nullable = true)
 |-- Species: string (nullable = true)



Número de linhas

In [10]:
df_iris.count()

150

Importação dos pacotes para trabalhar com vetores

In [11]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler, OneHotEncoder, StringIndexer

Removendo as linhas com valores nulos

In [12]:
df_iris = df_iris.na.drop()
df_iris.show()

+-------------+------------+-------------+------------+-----------+
|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|    Species|
+-------------+------------+-------------+------------+-----------+
|          5.1|         3.5|          1.4|         0.2|Iris-setosa|
|          4.9|         3.0|          1.4|         0.2|Iris-setosa|
|          4.7|         3.2|          1.3|         0.2|Iris-setosa|
|          4.6|         3.1|          1.5|         0.2|Iris-setosa|
|          5.0|         3.6|          1.4|         0.2|Iris-setosa|
|          5.4|         3.9|          1.7|         0.4|Iris-setosa|
|          4.6|         3.4|          1.4|         0.3|Iris-setosa|
|          5.0|         3.4|          1.5|         0.2|Iris-setosa|
|          4.4|         2.9|          1.4|         0.2|Iris-setosa|
|          4.9|         3.1|          1.5|         0.1|Iris-setosa|
|          5.4|         3.7|          1.5|         0.2|Iris-setosa|
|          4.8|         3.4|          1.6|      

Não há valores nulos, o número de linhas permanece o mesmo.

In [13]:
df_iris.count()

150

Transformação de atributos categóricos

In [14]:
label_index = StringIndexer(inputCol="Species", outputCol="indexedLabel")
model_label = label_index.fit(df_iris)
df_iris = model_label.transform(df_iris)

Montando o vetor de features

In [15]:
assembler = VectorAssembler(inputCols=["indexedLabel", "SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"],outputCol="features")
df_features = assembler.transform(df_iris)
df_features.show(20, False)

+-------------+------------+-------------+------------+-----------+------------+---------------------+
|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|Species    |indexedLabel|features             |
+-------------+------------+-------------+------------+-----------+------------+---------------------+
|5.1          |3.5         |1.4          |0.2         |Iris-setosa|0.0         |[0.0,5.1,3.5,1.4,0.2]|
|4.9          |3.0         |1.4          |0.2         |Iris-setosa|0.0         |[0.0,4.9,3.0,1.4,0.2]|
|4.7          |3.2         |1.3          |0.2         |Iris-setosa|0.0         |[0.0,4.7,3.2,1.3,0.2]|
|4.6          |3.1         |1.5          |0.2         |Iris-setosa|0.0         |[0.0,4.6,3.1,1.5,0.2]|
|5.0          |3.6         |1.4          |0.2         |Iris-setosa|0.0         |[0.0,5.0,3.6,1.4,0.2]|
|5.4          |3.9         |1.7          |0.4         |Iris-setosa|0.0         |[0.0,5.4,3.9,1.7,0.4]|
|4.6          |3.4         |1.4          |0.3         |Iris-setosa|0.0   

Seleção dos atributos que interessam para o modelo

In [16]:
df_features = df_features.select(df_features.indexedLabel, df_features.features)
df_features.show(20, False)

+------------+---------------------+
|indexedLabel|features             |
+------------+---------------------+
|0.0         |[0.0,5.1,3.5,1.4,0.2]|
|0.0         |[0.0,4.9,3.0,1.4,0.2]|
|0.0         |[0.0,4.7,3.2,1.3,0.2]|
|0.0         |[0.0,4.6,3.1,1.5,0.2]|
|0.0         |[0.0,5.0,3.6,1.4,0.2]|
|0.0         |[0.0,5.4,3.9,1.7,0.4]|
|0.0         |[0.0,4.6,3.4,1.4,0.3]|
|0.0         |[0.0,5.0,3.4,1.5,0.2]|
|0.0         |[0.0,4.4,2.9,1.4,0.2]|
|0.0         |[0.0,4.9,3.1,1.5,0.1]|
|0.0         |[0.0,5.4,3.7,1.5,0.2]|
|0.0         |[0.0,4.8,3.4,1.6,0.2]|
|0.0         |[0.0,4.8,3.0,1.4,0.1]|
|0.0         |[0.0,4.3,3.0,1.1,0.1]|
|0.0         |[0.0,5.8,4.0,1.2,0.2]|
|0.0         |[0.0,5.7,4.4,1.5,0.4]|
|0.0         |[0.0,5.4,3.9,1.3,0.4]|
|0.0         |[0.0,5.1,3.5,1.4,0.3]|
|0.0         |[0.0,5.7,3.8,1.7,0.3]|
|0.0         |[0.0,5.1,3.8,1.5,0.3]|
+------------+---------------------+
only showing top 20 rows



Renomeando a coluna Species para o rotulo label

In [17]:
df_novo = df_features.withColumnRenamed("indexedLabel", "label")
df_novo.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|[0.0,5.1,3.5,1.4,...|
|  0.0|[0.0,4.9,3.0,1.4,...|
|  0.0|[0.0,4.7,3.2,1.3,...|
|  0.0|[0.0,4.6,3.1,1.5,...|
|  0.0|[0.0,5.0,3.6,1.4,...|
|  0.0|[0.0,5.4,3.9,1.7,...|
|  0.0|[0.0,4.6,3.4,1.4,...|
|  0.0|[0.0,5.0,3.4,1.5,...|
|  0.0|[0.0,4.4,2.9,1.4,...|
|  0.0|[0.0,4.9,3.1,1.5,...|
|  0.0|[0.0,5.4,3.7,1.5,...|
|  0.0|[0.0,4.8,3.4,1.6,...|
|  0.0|[0.0,4.8,3.0,1.4,...|
|  0.0|[0.0,4.3,3.0,1.1,...|
|  0.0|[0.0,5.8,4.0,1.2,...|
|  0.0|[0.0,5.7,4.4,1.5,...|
|  0.0|[0.0,5.4,3.9,1.3,...|
|  0.0|[0.0,5.1,3.5,1.4,...|
|  0.0|[0.0,5.7,3.8,1.7,...|
|  0.0|[0.0,5.1,3.8,1.5,...|
+-----+--------------------+
only showing top 20 rows



Divisão da base nos conjuntos de treino e teste

In [18]:
# 70% para treinamento e 30% para teste
(trainingData, testData) = df_novo.randomSplit([0.7, 0.3])

print(trainingData.count(), testData.count())

102 48


In [19]:
trainingData.show(5)
testData.show(5)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|[0.0,4.3,3.0,1.1,...|
|  0.0|[0.0,4.4,2.9,1.4,...|
|  0.0|[0.0,4.4,3.0,1.3,...|
|  0.0|[0.0,4.4,3.2,1.3,...|
|  0.0|[0.0,4.5,2.3,1.3,...|
+-----+--------------------+
only showing top 5 rows

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|[0.0,4.7,3.2,1.6,...|
|  0.0|[0.0,4.8,3.0,1.4,...|
|  0.0|[0.0,4.8,3.0,1.4,...|
|  0.0|[0.0,4.8,3.4,1.6,...|
|  0.0|[0.0,4.9,3.1,1.5,...|
+-----+--------------------+
only showing top 5 rows



Treinamento de um modelo utilizando-se Regressão Logística

In [20]:
from pyspark.ml.classification import LogisticRegression 
#from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [21]:
# Cria o estimator para o modelo de regressão logística 
lr = LogisticRegression()

In [22]:
# Cria o transformer para o modelo de regressão logística 
lrModelo = lr.fit(trainingData)

In [23]:
# Mostra resultados de acurácia, verdadeiros positivos, falsos positivos e precisão
resultado = lrModelo.summary
print("Acurácia: %s" % str(resultado.accuracy))
print("Verdadeiros positivos: %s" % str(resultado.truePositiveRateByLabel))
print("Falsos positivos: %s" % str(resultado.falsePositiveRateByLabel))
print("Precisão: %s" %str(resultado.precisionByLabel))

Acurácia: 1.0
Verdadeiros positivos: [1.0, 1.0, 1.0]
Falsos positivos: [0.0, 0.0, 0.0]
Precisão: [1.0, 1.0, 1.0]


In [26]:
# Utiliza base de teste para modelo 
df_resultado = lrModelo.transform(testData)

# Mostra predições para cada label
df_resultado.show(48)

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|[0.0,4.7,3.2,1.6,...|[36.9699172518665...|[0.99999999999883...|       0.0|
|  0.0|[0.0,4.8,3.0,1.4,...|[36.9461187503628...|[0.99999999999849...|       0.0|
|  0.0|[0.0,4.8,3.0,1.4,...|[34.5002554209369...|[0.99999999995038...|       0.0|
|  0.0|[0.0,4.8,3.4,1.6,...|[38.5358887377238...|[0.99999999999989...|       0.0|
|  0.0|[0.0,4.9,3.1,1.5,...|[37.0263150019825...|[0.99999999999872...|       0.0|
|  0.0|[0.0,4.9,3.1,1.5,...|[37.0263150019825...|[0.99999999999872...|       0.0|
|  0.0|[0.0,5.0,3.0,1.6,...|[33.9554955487062...|[0.99999999987675...|       0.0|
|  0.0|[0.0,5.0,3.5,1.3,...|[39.1179736268891...|[0.99999999999995...|       0.0|
|  0.0|[0.0,5.0,3.5,1.6,...|[33.8839789903121...|[0.99999999992353...|       0.0|
|  0.0|[0.0,5.1,

In [27]:
from sklearn.metrics import classification_report, confusion_matrix

y_true = df_resultado.select(['label']).collect()
y_pred = df_resultado.select(['prediction']).collect()

from sklearn.metrics import classification_report, confusion_matrix

# Imprime relatório com dados de precisão, recall, f1-score
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        15
         1.0       1.00      1.00      1.00        15
         2.0       1.00      1.00      1.00        18

    accuracy                           1.00        48
   macro avg       1.00      1.00      1.00        48
weighted avg       1.00      1.00      1.00        48



### Conclusão

O modelo treinado teve uma acurácia e precisão de 100% para os valores preditos. 

Logicamente, a base iris.csv é uma base pequena com poucas instâncias e sem valores faltantes ou nulos o que torna o seu pré-processamento (limpeza e seleção dos dados, imputação de valores faltantes, etc) muito pequeno ou nulo.

No entanto, sua facilidade propicia um bom entendimento do funcionamento de um algoritmo de machine learning. Esta foi a minha ideia quando escolhi este dataset, entender e compreender algumas tecnicas de machine learning e alguns conceitos que avaliam a performance de um modelo (acurácia, precisão, etc).